# Song Track Database

This project generate an informative database leveraging open source web APIs as well as information scraped from online. 
Song track information including song names and artist names are scraped from Billboard year-end top 100 hit song charts from year 2010 to year 2019. Using this information, we send requests to Spotify APIs to fetch quantified audio analysis features provided by Spotify including popularity scores, acousticness, danceability, liveness, etc and to MusixMatch API to get song lyrics. All data is stored in JSON format in MongoDB which is more flexible and efficient.

In [25]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import re
from collections import OrderedDict
import numpy as np
import time
import os
from os import listdir
import pymongo
from pprint import pprint
from pymongo import MongoClient

#### **1. Scrape Top 100 Songs from Year 2010 to Year 2019**

In [ ]:
# Save Year-End Chart Pages
for year in range(2010,2020):
    billboard_URL = 'https://www.billboard.com/charts/year-end/'+str(year)+'/hot-100-songs'
    page = requests.get(url=billboard_URL)
    soup = bs(page.content, 'html.parser')
    f = open("Top 100 Songs Year"+str(year)+".htm","w", encoding="utf-8")
    f.write(str(soup))
    f.close()
    time.sleep(10)

In [5]:
# Loops through the pages downloaded, opens and parses them into an object. Identify and select relevant features.
all_songs_info = pd.DataFrame(columns=['Year','Rank', 'Song Name', 'Artist', 'Lyrics', 
                                       'album_name', 'album_release_date', 'album_total_tracks', 
                                       'track_popularity', 'track_duration_ms', 'track_available_markets', 'track_explicit',
                                       'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 
                                       'loudness', 'speechiness', 'tempo', 'valence',
                                       'Artist Birthday'])
for year in range(2010,2020):
    f = open(os.path.join(os.getcwd(), 'Top 100 Songs Year' + str(year) + '.htm'),"r",encoding="utf-8")
    soup = bs(f, 'html.parser')
    top_100_list = soup.find_all('article', {'class': 'ye-chart-item'})
    for song in top_100_list:
      song_rank = song.find('div', {'class': 'ye-chart-item__rank'}).text
      song_name = song.find('div', {'class': 'ye-chart-item__title'}).text
      song_artist = song.find('div', {'class': 'ye-chart-item__artist'}).text
      song_info = {'Year': year, 'Rank': song_rank, 'Song Name': song_name, 'Artist': song_artist}
      all_songs_info = all_songs_info.append(song_info, ignore_index=True)

In [6]:
# Data Cleaning: Remove all "\n"
all_songs_info["Rank"] = all_songs_info["Rank"].apply(lambda x: re.sub(r"\n","",str(x)))
all_songs_info["Song Name"] = all_songs_info["Song Name"].apply(lambda x: re.sub(r"\n","",str(x)))
all_songs_info["Artist"] = all_songs_info["Artist"].apply(lambda x: re.sub(r"\n","",str(x)))

In [9]:
all_songs_info.head()

,Year,Rank,Song Name,Artist,Lyrics,album_name,album_release_date,album_total_tracks,track_popularity,track_duration_ms,...,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,Artist Birthday
0,2010,1,TiK ToK,Ke$ha,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,2,Need You Now,Lady Antebellum,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,3,"Hey, Soul Sister",Train,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,4,California Gurls,Katy Perry Featuring Snoop Dogg,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,5,OMG,Usher Featuring will.i.am,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### **2. Use MusixMatch API to Fetch Top 100 Song Lyrics**

In [10]:
musixmatch_URL = "https://api.musixmatch.com/ws/1.1/matcher.lyrics.get"
API_KEY = '69abb0cfc4e8bf54e3e04f6f3fadb782'   # Replace with your API key or mine will reach the limits if everyone use it

In [11]:
for index, song in all_songs_info.iterrows():
  PARAMS = {'apikey': API_KEY,
            'q_track': song['Song Name'], 
            'q_artist': song['Artist']} 
  response = requests.get(url=musixmatch_URL, params=PARAMS).json()
  if response['message']['header']['status_code'] == 200:
    song['Lyrics'] = response['message']['body']['lyrics']['lyrics_body']
  else:
    print("Can't find lyrics for song {} by {}".format(song['Song Name'], song['Artist']))

Can't find lyrics for song OMG by Usher Featuring will.i.am
Can't find lyrics for song DJ Got Us Fallin' In Love by Usher Featuring Pitbull
Can't find lyrics for song Take It Off by Ke$ha
Can't find lyrics for song DJ Got Us Fallin' In Love by Usher Featuring Pitbull
Can't find lyrics for song Don't You Wanna Stay by Jason Aldean With Kelly Clarkson
Can't find lyrics for song Backseat by New Boyz Featuring The Cataracs & Dev
Can't find lyrics for song Take A Little Ride by Jason Aldean
Can't find lyrics for song Highway Don't Care by Tim McGraw With Taylor Swift
Can't find lyrics for song #thatPOWER by will.i.am Featuring Justin Bieber
Can't find lyrics for song Loyal by Chris Brown Featuring Lil Wayne & French Montana Or Too $hort Or Tyga
Can't find lyrics for song Rake It Up by Yo Gotti Featuring Nicki Minaj
Can't find lyrics for song Swervin by A Boogie Wit da Hoodie Featuring 6ix9ine


In [13]:
all_songs_info.head()

,Year,Rank,Song Name,Artist,Lyrics,album_name,album_release_date,album_total_tracks,track_popularity,track_duration_ms,...,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,Artist Birthday
0,2010,1,TiK ToK,Ke$ha,Theres some freaks in the livin'room gettin it...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,2,Need You Now,Lady Antebellum,Picture perfect memories\nScattered all around...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,3,"Hey, Soul Sister",Train,"Hey, hey\nHey, hey, hey, hey\nHey, hey, hey, h...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,4,California Gurls,Katy Perry Featuring Snoop Dogg,Greetings loved ones\nLets take journey!\n\nI ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,5,OMG,Usher Featuring will.i.am,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Use SpotIfy API to Get General Song Profile and Audio Analysis Features**

In [19]:
# Connect to Spotify API
CLIENT_ID = '80ac2d8e054a4224a8b8e89be71d9efe'
CLIENT_SECRET = '8c29bd253adf44318a221ceed7842926'

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

In [15]:
# Audio Analysis Feature
def get_spotify_features(song_name, artist):
    #Search for Spofity song ID 
    songs=sp.search(q='track:'+song_name+' '+'artist:'+artist+'*' , type='track')
    items = songs['tracks']['items']
    try:
        track = items[0]
        song_id = str(track["id"])
    #Use song ID to get Song features
        track_features=sp.audio_features(song_id)
        features_to_df = np.array(track_features)[0]
        features_to_df["Song Name"] = song_name
        features_to_df["Artist"] = artist
        return(features_to_df)
    except:
        return {"Artist": artist,
                "Song Name": song_name}

In [16]:
for index, song in all_songs_info.iterrows():
    try:
        audio = get_spotify_features(song["Song Name"], song["Artist"])
        song["acousticness"] = audio["acousticness"]
        song["danceability"] = audio["danceability"]
        song["energy"] = audio["energy"]
        song["instrumentalness"] = audio["instrumentalness"]
        song["key"] = audio["key"]
        song["liveness"] = audio["liveness"]
        song["loudness"] = audio["loudness"]
        song["speechiness"] = audio["speechiness"]
        song["tempo"] = audio["tempo"]
        song["valence"] = audio["valence"]
    except:
        print("Can't find info for song {} by {}".format(song['Song Name'], song['Artist']))

Can't find info for song TiK ToK by Ke$ha
Can't find info for song California Gurls by Katy Perry Featuring Snoop Dogg
Can't find info for song OMG by Usher Featuring will.i.am
Can't find info for song Airplanes by B.o.B Featuring Hayley Williams
Can't find info for song Love The Way You Lie by Eminem Featuring Rihanna
Can't find info for song Break Your Heart by Taio Cruz Featuring Ludacris
Can't find info for song Nothin' On You by B.o.B Featuring Bruno Mars
Can't find info for song I Like It by Enrique Iglesias Featuring Pitbull
Can't find info for song BedRock by Young Money Featuring Lloyd
Can't find info for song Telephone by Lady Gaga Featuring Beyonce
Can't find info for song Empire State Of Mind by Jay-Z + Alicia Keys
Can't find info for song  Billionaire by Travie McCoy Featuring Bruno Mars
Can't find info for song Sexy Chick by David Guetta Featuring Akon
Can't find info for song Say Aah by Trey Songz Featuring Fabolous
Can't find info for song Like A G6 by Far*East Movement

Can't find info for song Feel This Moment by Pitbull Featuring Christina Aguilera
Can't find info for song Love Me by Lil Wayne Featuring Drake & Future
Can't find info for song F**kin Problems by A$AP Rocky Featuring Drake, 2 Chainz & Kendrick Lamar
Can't find info for song Beauty And A Beat by Justin Bieber Featuring Nicki Minaj
Can't find info for song Same Love by Macklemore & Ryan Lewis Featuring Mary Lambert
Can't find info for song Sweet Nothing by Calvin Harris Featuring Florence Welch
Can't find info for song Summertime Sadness by Lana Del Rey & Cedric Gervais
Can't find info for song Power Trip by J. Cole Featuring Miguel
Can't find info for song Girl On Fire  by Alicia Keys Featuring Nicki Minaj
Can't find info for song I Need Your Love by Calvin Harris Featuring Ellie Goulding
Can't find info for song Bad by Wale Featuring Tiara Thomas Or Rihanna
Can't find info for song Boys 'round Here by Blake Shelton Featuring Pistol Annies & Friends
Can't find info for song Highway Don

Can't find info for song Starboy by The Weeknd Featuring Daft Punk
Can't find info for song For Free by DJ Khaled Featuring Drake
Can't find info for song Never Be Like You by Flume Featuring Kai
Can't find info for song Close by Nick Jonas Featuring Tove Lo
Can't find info for song Down In The DM by Yo Gotti Featuring Nicki Minaj
Can't find info for song Can't Feel My Face by The Weeknd
Can't find info for song Side To Side by Ariana Grande Featuring Nicki Minaj
Can't find info for song Middle by DJ Snake Featuring Bipolar Sunshine
Can't find info for song Pop Style by Drake Featuring The Throne
Can't find info for song Lean On by Major Lazer & DJ Snake Featuring MO
Can't find info for song I Know What You Did Last Summer by Shawn Mendes & Camila Cabello
Can't find info for song No Limit by Usher Featuring Young Thug
Can't find info for song Cut It by O.T. Genasis Featuring Young Dolph
Can't find info for song All In My Head (Flex) by Fifth Harmony Featuring Fetty Wap
Can't find info 

Can't find info for song Con Calma by Daddy Yankee & Katy Perry Featuring Snow
Can't find info for song I Like It by Cardi B, Bad Bunny & J Balvin
Can't find info for song Swervin by A Boogie Wit da Hoodie Featuring 6ix9ine
Can't find info for song Baby by Lil Baby & DaBaby
Can't find info for song Clout by Offset Featuring Cardi B
Can't find info for song Love Lies by Khalid & Normani
Can't find info for song Cash Shit by Megan Thee Stallion Featuring DaBaby
Can't find info for song Beautiful by Bazzi Featuring Camila Cabello
Can't find info for song Boyfriend by Ariana Grande & Social House


In [20]:
# General Song Profile
for index, song in all_songs_info.iterrows():
  search_query = song['Song Name'] + " " + song['Artist']
  results = sp.search(search_query, limit=1, type='track')
  if results["tracks"]["items"]:
    track_info = results['tracks']['items'][0]
    album_info = track_info['album']
    song["album_name"] = album_info['name']
    song["album_release_date"] = album_info['release_date']
    song["album_total_tracks"] = album_info['total_tracks']
    song["track_popularity"] = track_info['popularity']
    song["track_duration_ms"] = track_info['duration_ms']
    song["track_available_markets"] = track_info['available_markets']
    song["track_explicit"] = track_info['explicit']
  else:
    print("Can't find info for song {} by {}".format(song['Song Name'], song['Artist']))

Can't find info for song California Gurls by Katy Perry Featuring Snoop Dogg
Can't find info for song OMG by Usher Featuring will.i.am
Can't find info for song  Billionaire by Travie McCoy Featuring Bruno Mars
Can't find info for song Say Aah by Trey Songz Featuring Fabolous
Can't find info for song Like A G6 by Far*East Movement Featuring Cataracs & Dev
Can't find info for song Blah Blah Blah by Ke$ha Featuring 3OH!3
Can't find info for song Bottoms Up by Trey Songz Featuring Nicki Minaj
Can't find info for song My Chick Bad by Ludacris Featuring Nicki Minaj
Can't find info for song Deuces by Chris Brown Featuring Tyga & Kevin McCall
Can't find info for song Forever by Drake Featuring Kanye West, Lil Wayne & Eminem
Can't find info for song All I Do Is Win by DJ Khaled Featuring T-Pain, Ludacris, Snoop Dogg & Rick Ross
Can't find info for song I Made It (Cash Money Heroes) by Kevin Rudolf Featuring Birdman, Jay Sean, & Lil Wayne
Can't find info for song Right Above It by Lil Wayne Feat

Can't find info for song Lifestyle by Rich Gang Featuring Young Thug & Rich Homie Quan
Can't find info for song New Flame by Chris Brown Featuring Usher & Rick Ross
Can't find info for song La La La by Naughty Boy Featuring Sam Smith
Can't find info for song Blurred Lines by Robin Thicke Featuring T.I. + Pharrell
Can't find info for song Can't Remember To Forget You by Shakira Featuring Rihanna
Can't find info for song No Mediocre by T.I. Featuring Iggy Azalea
Can't find info for song Believe Me by Lil Wayne Featuring Drake
Can't find info for song 23 by Mike WiLL Made-It Featuring Miley Cyrus, Wiz Khalifa & Juicy J
Can't find info for song White Walls by Macklemore & Ryan Lewis Featuring ScHoolboy Q & Hollis
Can't find info for song Studio by ScHoolboy Q Featuring BJ The Chicago Kid
Can't find info for song Uptown Funk! by Mark Ronson Featuring Bruno Mars
Can't find info for song Bad Blood by Taylor Swift Featuring Kendrick Lamar
Can't find info for song Lean On by Major Lazer & DJ Sn

Can't find info for song Lights Down Low by MAX Featuring gnash
Can't find info for song I Get The Bag by Gucci Mane Featuring Migos
Can't find info for song No Brainer by DJ Khaled Featuring Justin Bieber, Chance The Rapper & Quavo
retrying ...2secs
Can't find info for song Plain Jane by A$AP Ferg Featuring Nicki Minaj
Can't find info for song Sky Walker by Miguel Featuring Travis Scott
Can't find info for song 1-800-273-8255 by Logic Featuring Alessia Cara & Khalid
Can't find info for song Say Something by Justin Timberlake Featuring Chris Stapleton
Can't find info for song What Lovers Do by Maroon 5 Featuring SZA
Can't find info for song Mi Gente by J Balvin & Willy William Featuring Beyonce
Can't find info for song Old Town Road by Lil Nas X Featuring Billy Ray Cyrus
Can't find info for song Going Bad by Meek Mill Featuring Drake
Can't find info for song No Guidance by Chris Brown Featuring Drake
Can't find info for song Girls Like You by Maroon 5 Featuring Cardi B
Can't find info 

#### Store All information Fetched from Billboard, Spotify API, MusicxMatch API in the dataframe below

In [23]:
all_songs_info.head()

,Year,Rank,Song Name,Artist,Lyrics,album_name,album_release_date,album_total_tracks,track_popularity,track_duration_ms,...,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,Artist Birthday
0,2010,1,TiK ToK,Ke$ha,Theres some freaks in the livin'room gettin it...,Animal (Expanded Edition),2010-01-01,18,78,199693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,2,Need You Now,Lady Antebellum,Picture perfect memories\nScattered all around...,Need You Now,2010-01-01,11,68,277573,...,0.587,0.622,0.000636,4,0.2,-5.535,0.0303,107.943,0.231,NaN
2,2010,3,"Hey, Soul Sister",Train,"Hey, hey\nHey, hey, hey, hey\nHey, hey, hey, h...","Save Me, San Francisco (Golden Gate Edition)",2010-12-01,19,81,216773,...,0.673,0.886,0,1,0.0826,-4.44,0.0431,97.012,0.795,NaN
3,2010,4,California Gurls,Katy Perry Featuring Snoop Dogg,Greetings loved ones\nLets take journey!\n\nI ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,5,OMG,Usher Featuring will.i.am,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### **Save Data to MongoDB**

In [26]:
client = MongoClient()
db = client.MusicProfile
collection = db.General_AudioAnalysis
all_songs_info_dict = all_songs_info.to_dict('records')
collection.insert_many(all_songs_info_dict)
client.close()